In [1]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 50)

In [2]:
start_year = 18
end_year = 19
weeks = [str(i).rjust(2,'0') for i in range(1,14)]
data_urls = []
while start_year < 24:
    for week in weeks:
        if start_year == 23 and week > '02':
            break
        if start_year == 19 and week == '04':
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-Delays-20{start_year}{end_year}-P{week}.zip')
        elif start_year == 19 and week > '04':
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-delays-20{start_year}-{end_year}-P{week}.zip')
        elif start_year > 19:
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-delays-20{start_year}-{end_year}-P{week}.zip')
        else:
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-Delays-20{start_year}-{end_year}-P{week}.zip')
    start_year+=1
    end_year+=1

In [5]:
cols_to_rename = {
    #New name: old name
    'APPLICABLE_TIMETABLE_FLAG': 'APP_TIMETABLE_FLAG_AFF',
    'FINANCIAL_YEAR_PERIOD': 'FINANCIAL_YEAR_AND_PERIOD',
    'INCIDENT_RESP_TRAIN': 'INCIDENT_RESPONSIBLE_TRAIN',
    'NR_LOCATION_MANAGER': 'NETWORK_RAIL_LOCATION_MANAGER',
    'EVENT_TYPE': 'PERFORMANCE_EVENT_CODE',
    'PLANNED_DEST_GBTT_DATETIME':'PLANNED_DEST_GBTT_DATETIME_AFF',
    'PLANNED_DEST_LOCATION_CODE':'PLANNED_DEST_LOC_CODE_AFFECTED',
    'PLANNED_DEST_WTT_DATETIME':'PLANNED_DEST_WTT_DATETIME_AFF',
    'PLANNED_ORIGIN_GBTT_DATETIME':'PLANNED_ORIG_GBTT_DATETIME_AFF',
    'PLANNED_ORIGIN_LOCATION_CODE':'PLANNED_ORIG_LOC_CODE_AFF',
    'PLANNED_ORIGIN_WTT_DATETIME':'PLANNED_ORIG_WTT_DATETIME_AFF',
    'REACT_REASON':'REACTIONARY_REASON_CODE',
    'SERVICE_GROUP_CODE':'SERVICE_GROUP_CODE_AFFECTED',
    'TIMING_LOAD':'TIMING_LOAD_AFFECTED',
    'TRACTION_TYPE': 'TRACTION_TYPE_AFFECTED',
    'TRAILING_LOAD': 'TRAILING_LOAD_AFFECTED',
    'TRAIN_SCHEDULE_TYPE' : 'TRAIN_SCHEDULE_TYPE_AFFECTED',
    'TRAIN_SERVICE_CODE': 'TRAIN_SERVICE_CODE_AFFECTED',
    'UNIT_CLASS':'UNIT_CLASS_AFFECTED',
    'TOC_CODE' : 'OPERATOR_AFFECTED',
    'TRUST_TRAIN_ID': 'TRUST_TRAIN_ID_AFFECTED',
    'RESP_MANAGER' : 'RESPONSIBLE_MANAGER',
    'TRUST_TRAIN_ID_REACT' : 'REACT_TRAIN',
    'TRUST_TRAIN_ID_RESP' : 'RESP_TRAIN'
    }

In [82]:
for i in data_urls[:13]:
    df = pd.read_csv(i)
    print(df.iloc[-1])

FINANCIAL_YEAR_AND_PERIOD               2018/19_P01
ORIGIN_DEPARTURE_DATE                     28-APR-18
TRUST_TRAIN_ID_AFFECTED                  895Q57CU28
PLANNED_ORIG_LOC_CODE_AFF                     89472
PLANNED_ORIG_GBTT_DATETIME_AFF                  NaN
                                        ...        
END_STANOX                                    89472
EVENT_DATETIME                    28-APR-2018 17:13
PFPI_MINUTES                                    0.0
TRUST_TRAIN_ID_RESP                             NaN
TRUST_TRAIN_ID_REACT                            NaN
Name: 490317, Length: 39, dtype: object
FINANCIAL_YEAR_AND_PERIOD               2018/19_P02
ORIGIN_DEPARTURE_DATE                     26-MAY-18
TRUST_TRAIN_ID_AFFECTED                  899Y70M826
PLANNED_ORIG_LOC_CODE_AFF                     89351
PLANNED_ORIG_GBTT_DATETIME_AFF    26-MAY-2018 22:52
                                        ...        
END_STANOX                                    89351
EVENT_DATETIME          

In [153]:
year = 2023

In [154]:
import numpy as np
stacked = pd.DataFrame()
for i in data_urls[65:]:
    df = pd.read_csv(i)
    columns = [col.upper() for col in list(df.columns)]
    df.columns = columns
    df.rename(columns = cols_to_rename, inplace = True)
    if df.iloc[-1]['ORIGIN_DEPARTURE_DATE'] == np.nan:
        df = df.iloc[:-1]
    df = df[df['OPERATOR_AFFECTED'] == 'EK']
#     df = df[(df['PERFORMANCE_EVENT_CODE'] == 'A') | (df['PERFORMANCE_EVENT_CODE'] == 'M')]
    
    df['ORIGIN_DEPARTURE_DATE'] = pd.to_datetime(df['ORIGIN_DEPARTURE_DATE'], format = '%d-%b-%y' if len(df.iloc[0]['ORIGIN_DEPARTURE_DATE'])==9 else '%d/%m/%Y %H:%M')
    df['PLANNED_ORIG_GBTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_ORIG_GBTT_DATETIME_AFF'])
    df['PLANNED_ORIG_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_ORIG_WTT_DATETIME_AFF'])
    df['PLANNED_DEST_GBTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_DEST_GBTT_DATETIME_AFF'])
    df['PLANNED_DEST_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_DEST_WTT_DATETIME_AFF'])
    df['INCIDENT_CREATE_DATE'] = pd.to_datetime(df['INCIDENT_CREATE_DATE'])
    df['INCIDENT_START_DATETIME'] = pd.to_datetime(df['INCIDENT_START_DATETIME'])
    df['INCIDENT_END_DATETIME'] = pd.to_datetime(df['INCIDENT_END_DATETIME'])
    df['EVENT_DATETIME'] = pd.to_datetime(df['EVENT_DATETIME'])
    df['TRAIN_SERVICE_CODE_AFFECTED'] = df['TRAIN_SERVICE_CODE_AFFECTED'].astype('int64', errors = 'ignore')
    df['UNIT_CLASS_AFFECTED'] = df['UNIT_CLASS_AFFECTED'].astype('str')
    if 'NON_PFPI_MINUTES' in df.columns:
        df['PFPI_MINUTES'] = df['PFPI_MINUTES'] + df['NON_PFPI_MINUTES']
        df = df.drop(columns = ['NON_PFPI_MINUTES'])

    #Takes care of pointless column in 2020/21 data
    if 'UNNAMED: 40' in df.columns:
        df = df.drop(columns = ['UNNAMED: 40'])

    if 'UNNAMED: 41' in df.columns:
        df = df.drop(columns = ['UNNAMED: 41'])
    print(f'Dataset: {i[-15:]} - shape is {df.shape}')
    stacked = pd.concat([stacked, df], ignore_index=True)

Dataset: 2023-24-P01.zip - shape is (10362, 39)
Dataset: 2023-24-P02.zip - shape is (12940, 39)


In [155]:
stacked.to_csv(f'../data/EK_{year}.csv', mode = 'w', index = False)

In [156]:
stacked

,FINANCIAL_YEAR_AND_PERIOD,ORIGIN_DEPARTURE_DATE,TRUST_TRAIN_ID_AFFECTED,PLANNED_ORIG_LOC_CODE_AFF,PLANNED_ORIG_WTT_DATETIME_AFF,PLANNED_ORIG_GBTT_DATETIME_AFF,PLANNED_DEST_LOC_CODE_AFFECTED,PLANNED_DEST_WTT_DATETIME_AFF,PLANNED_DEST_GBTT_DATETIME_AFF,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,OPERATOR_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,TRAIN_SCHEDULE_TYPE_AFFECTED,TRACTION_TYPE_AFFECTED,TRAILING_LOAD_AFFECTED,TIMING_LOAD_AFFECTED,UNIT_CLASS_AFFECTED,INCIDENT_NUMBER,INCIDENT_CREATE_DATE,INCIDENT_START_DATETIME,INCIDENT_END_DATETIME,SECTION_CODE,NETWORK_RAIL_LOCATION_MANAGER,RESPONSIBLE_MANAGER,INCIDENT_REASON,ATTRIBUTION_STATUS,INCIDENT_EQUIPMENT,INCIDENT_DESCRIPTION,REACTIONARY_REASON_CODE,INCIDENT_RESPONSIBLE_TRAIN,RESP_TRAIN,REACT_TRAIN,PERFORMANCE_EVENT_CODE,START_STANOX,END_STANOX,EVENT_DATETIME,PFPI_MINUTES
0,2023/24_P01,2023-04-01,502V02M601,50401,2023-04-01 06:11:00,2023-04-01 06:11:00,50392,2023-04-01 06:20:00,2023-04-01 06:20:00,21237001,EK05,EK,SA,Y,LTP,EMU,NaN,NaN,710.0,396590,2023-04-01,2023-04-01 05:00:00,2023-04-01 13:21:00,50334:50423,OQHN,QQHE,QB,Attribution Agreed,NaN,IFD-GDP ITEM4 SUPP2 DIVERT,QB,NaN,NaN,NaN,C,50401,50401,2023-04-01 06:11:00,45.0
1,2023/24_P01,2023-04-01,502V03M601,50392,2023-04-01 06:25:00,2023-04-01 06:25:00,50401,2023-04-01 06:34:00,2023-04-01 06:34:00,21237001,EK05,EK,SA,Y,LTP,EMU,NaN,NaN,710.0,396590,2023-04-01,2023-04-01 05:00:00,2023-04-01 13:21:00,50334:50423,OQHN,QQHE,QB,Attribution Agreed,NaN,IFD-GDP ITEM4 SUPP2 DIVERT,QB,NaN,NaN,NaN,C,50392,50392,2023-04-01 06:25:00,45.0
2,2023/24_P01,2023-04-01,505V02M501,50422,2023-04-01 05:24:00,NaT,50401,2023-04-01 06:01:00,NaT,21252001,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,710.0,396287,2023-04-01,2023-04-01 00:01:00,2023-04-01 08:18:00,52741,OQHN,OQHU,OU,Attribution Agreed,NaN,LST ANGLIA ECS DELAYS 01/04,NaN,NaN,NaN,NaN,M,50422,50422,2023-04-01 06:06:00,42.0
3,2023/24_P01,2023-04-01,512D01M501,51815,2023-04-01 05:17:00,2023-04-01 05:17:00,52741,2023-04-01 05:55:00,2023-04-01 05:56:00,21234001,EK04,EK,SA,Y,LTP,EMU,NaN,NaN,710.0,396604,2023-04-01,2023-04-01 05:20:00,2023-04-01 13:28:00,51815,OQHP,CQHL,OH,Attribution Agreed,NaN,2D01 ARS CHN,NaN,2D01,512D01M501,NaN,M,51815,51815,2023-04-01 05:20:00,3.0
4,2023/24_P01,2023-04-01,512J02M601,51533,2023-04-01 06:18:00,2023-04-01 06:18:00,51541,2023-04-01 06:27:00,2023-04-01 06:29:00,21921000,EK01,EK,SA,Y,(V)STP Overlay,EMU,NaN,NaN,710.0,396362,2023-04-01,2023-04-01 01:18:00,2023-04-01 09:31:00,36322:36501,OQFR,APEA,AC,Attribution Agreed,NaN,6L47 TRAIN PREP DITTON,YB,366L471831,366L471831,366L471831,M,51533,51531,2023-04-01 06:25:00,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23297,2023/24_P02,2023-05-27,889F65M727,88392,2023-05-27 21:57:00,2023-05-27 21:57:00,52074,2023-05-27 22:19:00,2023-05-27 22:19:00,22218000,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,378.0,533095,2023-05-27,2023-05-27 17:42:00,2023-05-28 01:50:00,52084:87568,OQYA,VEK1,VA,Attribution Agreed,NaN,9C47 DISORDER ZLW ZCW,YI,9C47,529C47MU27,529E64M727,C,88392,88392,2023-05-27 21:57:00,0.0
23298,2023/24_P02,2023-05-27,889F67M827,88392,2023-05-27 22:27:00,2023-05-27 22:27:00,52074,2023-05-27 22:49:00,2023-05-27 22:49:00,22218000,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,378.0,533543,2023-05-27,2023-05-27 21:43:00,2023-05-28 05:54:00,87603:87643,OQM6,VEKA,VH,Attribution Agreed,NaN,9C62 PASS COMM NXG NWD,YD,9C62,529C62MZ27,879D67M827,M,88392,87568,2023-05-27 22:36:00,3.0
23299,2023/24_P02,2023-05-27,889F68M827,88392,2023-05-27 22:42:00,2023-05-27 22:42:00,52074,2023-05-27 23:04:00,2023-05-27 23:04:00,22218000,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,378.0,476572,2023-05-04,2023-04-30 00:01:00,2023-05-27 23:59:00,87643:87603,OQM6,QQA0,QA,Attribution Agreed,260124,BCY NXG ARL SCHEDULE PERIOD 2,YD,NaN,NaN,879D68M827,M,88392,87568,2023-05-27 22:49:00,1.0
23300,2023/24_P02,2023-05-27,889F68M827,88392,2023-05-27 22:42:00,2023-05-27 22:42:00,52074,20

In [157]:
lookup_csv_path = "../data/Stanox-Station-Lookup.csv"
mapping_df = pd.read_csv(lookup_csv_path, sep=';')
mapping_df = mapping_df[['Stanox', 'Latitude', 'Longitude', 'Station Name']]

In [158]:
origin_geo_locations = pd.merge(stacked,
                        mapping_df,
                        left_on ='START_STANOX',
                        right_on = 'Stanox' ,
                        how ='left')
    # drop observation w/o origin latitude and longitude, drop redundant columns
# origin_geo_locations = origin_geo_locations.dropna(subset=['Latitude', 'Longitude'])
origin_geo_locations = origin_geo_locations.drop(columns = ['Stanox'])
new_column_names = {'Latitude': 'Lat_OR', 'Longitude': 'Lon_OR', 'Station Name': 'STATION_OR'}
origin_geo_locations.rename(columns=new_column_names, inplace=True)

# merge mapping dataset with the train delays data to get latitude and longitude for destination stanox code
origin_dest_locations = pd.merge(origin_geo_locations,
                  mapping_df,
                  left_on ='END_STANOX',
                  right_on = 'Stanox' ,
                  how ='left')
# drop observation w/o destination latitude and longitude, drop redundant columns
# origin_dest_locations = origin_dest_locations.dropna(subset=['Latitude', 'Longitude'])
origin_dest_locations = origin_dest_locations.drop(columns = ['Stanox'])
new_column_names = {'Latitude': 'Lat_DES', 'Longitude': 'Lon_DES', 'Station Name': 'STATION_DES'}
origin_dest_locations.rename(columns=new_column_names, inplace=True)


In [159]:
reasons_csv = pd.read_excel('https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/Reference%20Files/Transpareny%20Page%20Attribution%20Glossary.xlsx', sheet_name='Incident Reason')[['Incident Reason', 'Incident Reason Description']]
reasons_csv.rename(columns={'Incident Reason': 'INCIDENT_REASON', 'Incident Reason Description': 'INCIDENT_REASON_DESCRIPTION'}, inplace=True)

In [160]:
EK_merged = origin_dest_locations.merge(reasons_csv, on='INCIDENT_REASON')
EK_merged.to_csv(f'../data/EK_{year}_merged.csv', index=False)

In [161]:
EK_merged

,FINANCIAL_YEAR_AND_PERIOD,ORIGIN_DEPARTURE_DATE,TRUST_TRAIN_ID_AFFECTED,PLANNED_ORIG_LOC_CODE_AFF,PLANNED_ORIG_WTT_DATETIME_AFF,PLANNED_ORIG_GBTT_DATETIME_AFF,PLANNED_DEST_LOC_CODE_AFFECTED,PLANNED_DEST_WTT_DATETIME_AFF,PLANNED_DEST_GBTT_DATETIME_AFF,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,OPERATOR_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,TRAIN_SCHEDULE_TYPE_AFFECTED,TRACTION_TYPE_AFFECTED,TRAILING_LOAD_AFFECTED,TIMING_LOAD_AFFECTED,UNIT_CLASS_AFFECTED,INCIDENT_NUMBER,INCIDENT_CREATE_DATE,INCIDENT_START_DATETIME,INCIDENT_END_DATETIME,SECTION_CODE,NETWORK_RAIL_LOCATION_MANAGER,RESPONSIBLE_MANAGER,INCIDENT_REASON,ATTRIBUTION_STATUS,INCIDENT_EQUIPMENT,INCIDENT_DESCRIPTION,REACTIONARY_REASON_CODE,INCIDENT_RESPONSIBLE_TRAIN,RESP_TRAIN,REACT_TRAIN,PERFORMANCE_EVENT_CODE,START_STANOX,END_STANOX,EVENT_DATETIME,PFPI_MINUTES,Lat_OR,Lon_OR,STATION_OR,Lat_DES,Lon_DES,STATION_DES,INCIDENT_REASON_DESCRIPTION
0,2023/24_P01,2023-04-01,502V02M601,50401,2023-04-01 06:11:00,2023-04-01 06:11:00,50392,2023-04-01 06:20:00,2023-04-01 06:20:00,21237001,EK05,EK,SA,Y,LTP,EMU,NaN,NaN,710.0,396590,2023-04-01,2023-04-01 05:00:00,2023-04-01 13:21:00,50334:50423,OQHN,QQHE,QB,Attribution Agreed,NaN,IFD-GDP ITEM4 SUPP2 DIVERT,QB,NaN,NaN,NaN,C,50401,50401,2023-04-01 06:11:00,45.0,51.574830,0.183264,Romford Rail Station,51.574830,0.183264,Romford Rail Station,Planned engineering work - diversion/SLW not t...
1,2023/24_P01,2023-04-01,502V03M601,50392,2023-04-01 06:25:00,2023-04-01 06:25:00,50401,2023-04-01 06:34:00,2023-04-01 06:34:00,21237001,EK05,EK,SA,Y,LTP,EMU,NaN,NaN,710.0,396590,2023-04-01,2023-04-01 05:00:00,2023-04-01 13:21:00,50334:50423,OQHN,QQHE,QB,Attribution Agreed,NaN,IFD-GDP ITEM4 SUPP2 DIVERT,QB,NaN,NaN,NaN,C,50392,50392,2023-04-01 06:25:00,45.0,51.559108,0.250912,Upminster Rail Station,51.559108,0.250912,Upminster Rail Station,Planned engineering work - diversion/SLW not t...
2,2023/24_P01,2023-04-05,722C73M905,72000,2023-04-05 23:30:00,2023-04-05 23:30:00,72410,2023-04-06 00:21:00,2023-04-06 00:23:00,22216000,EK02,EK,WD,Y,LTP,EMU,NaN,NaN,710.0,408041,2023-04-05,2023-04-05 23:00:00,2023-04-06 05:00:00,72410:72268,OQNE,QQN3,QB,Attribution Agreed,ITEM 53,WON 1 ITEM 53 CMD WMB,NaN,NaN,NaN,NaN,M,72311,72410,2023-04-06 00:25:00,4.0,51.533966,-0.204960,Queens Park (London) Rail Station,51.528136,-0.133898,London Euston Rail Station,Planned engineering work - diversion/SLW not t...
3,2023/24_P01,2023-04-05,873M71M905,87219,2023-04-05 23:55:00,NaT,72116,2023-04-06 00:20:00,NaT,22216001,EK99,EK,WD,Y,LTP,EMU,NaN,NaN,378.0,408041,2023-04-05,2023-04-05 23:00:00,2023-04-06 05:00:00,72410:72268,OQNE,QQN3,QB,Attribution Agreed,ITEM 53,WON 1 ITEM 53 CMD WMB,YB,NaN,NaN,720A96M905,M,72268,72116,2023-04-06 00:24:00,4.0,NaN,NaN,'',NaN,NaN,'',Planned engineering work - diversion/SLW not t...
4,2023/24_P02,2023-05-03,525M72M903,52226,2023-05-03 23:03:00,NaT,72410,2023-05-04 00:15:00,NaT,22216001,EK99,EK,WD,Y,LTP,EMU,NaN,NaN,378.0,475685,2023-05-04,2023-05-04 00:10:00,2023-05-04 05:50:00,72311:72410,OQNE,QQN3,QB,Attribution Agreed,ITEM 59,WON 5 ITEM 59 EUS,NaN,NaN,NaN,NaN,M,72311,72410,2023-05-04 00:19:00,3.0,51.533966,-0.204960,Queens Park (London) Rail Station,51.528136,-0.133898,London Euston Rail Station,Planned engineering work - diversion/SLW not t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23297,2023/24_P02,2023-05-25,872N46MS25,87132,2023-05-25 16:08:00,2023-05-25 16:08:00,52226,2023-05-25 17:09:00,2023-05-25 17:10:00,22214000,EK01,EK,WD,Y,LTP,EMU,NaN,NaN,378.0,527250,2023-05-25,2023-05-25 13:51:00,2023-05-25 22:34:00,70092,OQNA,AXHA,AH,Attribution Agreed,66427,4L48 LATE START DIRFTRR,YO,704L48CN25,704L48CN25,522N04MU25,M,52045,52226,2023-05-25 17:13:00,4.0,NaN,NaN,'',51.541895,-0.003369,Stratford (London) Rail Station,Customer or off Network Rail network yard equi...
23298,2023/24_P02,2023